In [4]:
import pandas as pd

df = pd.read_csv('..\data\KDDTrain+_20Percent.txt')

print(pd)


<module 'pandas' from 'c:\\Users\\caioc\\miniconda3\\envs\\ml-ids\\Lib\\site-packages\\pandas\\__init__.py'>
